# Data preprocess 2

In [1]:
# Preprocess 3 Select Interested Data By Question Id
# ---------------------------------------------

import pandas

## Load trimmed data
Answer = pandas.read_sql('Answer_trimmed', "sqlite:///data/QAT_trimmed.db", index_col='index')
Question = pandas.read_sql('Question_trimmed', "sqlite:///data/QAT_trimmed.db", index_col='index')
print("Load Data")

Load Data


In [2]:
## Sort data by CreationDate_to_datetime & Score
Answer_sorted_score = Answer.sort_values(by=['ParentId','Score','CreationDate_to_datetime'],ascending=[1,0,1])
Answer_sorted_time = Answer.sort_values(by=['ParentId','CreationDate_to_datetime'])
Question_sorted_Id = Question.sort_values(by=['Id'])
print("Sort data")

Sort data


In [3]:
## Select interested data, and put into Question dataframe

## Create iterator
time_gen = Answer_sorted_time.iterrows()
score_gen = Answer_sorted_score.iterrows()
count = 0
next_score_data = next(score_gen)
next_time_data = next(time_gen)

for idx, row in Question_sorted_Id.iterrows():
    ## Put first data into Question dataframe by row
    Question_sorted_Id.at[idx,'BestAnsTime'] = next_score_data[1]['CreationDate_to_datetime']
    Question_sorted_Id.at[idx,'FirstAnsTime'] = next_time_data[1]['CreationDate_to_datetime']
    next_score_data = next(score_gen)
    next_time_data = next(time_gen)

    ## Skip other data
    while(next_score_data[1]['ParentId'] == row['Id']):
        try:
            next_score_data = next(score_gen)
            next_time_data = next(time_gen)
        except StopIteration:
            print('\nfinished')
            break
    if count % 10000 == 0:
        print('#', end='')
    count+=1


###############################################################################################################
finished


In [4]:
## Store data to another DB
Question_sorted_Id = Question_sorted_Id.drop(columns=['OwnerUserId','ClosedDate'])
Question_sorted_Id.to_sql('Question', "sqlite:///data/QAT_final.db",  if_exists = "replace")
print("Store Final Data to DB")
# ---------------------------------------------

Store Final Data to DB
